# SchNet S2EF training example

The purpose of this notebook is to demonstrate some of the basics of the Open Catalyst Project's (OCP) codebase and data. In this example, we will train a schnet model for predicting the energy and forces of a given structure (S2EF task). First, ensure you have installed the OCP ocp repo and all the dependencies according to the [README](https://github.com/Open-Catalyst-Project/ocp/blob/master/README.md).

Disclaimer: This notebook is for tutorial purposes, it is unlikely it will be practical to train baseline models on our larger datasets using this format. As a next step, we recommend trying the command line examples. 

## Imports

In [18]:
import torch
import os
from ocpmodels.trainers import ForcesTrainer
from ocpmodels import models

In [9]:
# a simple sanity check that a GPU is available
if torch.cuda.is_available():
    print("True")
else:
    print("False")

True


## The essential steps for training an OCP model

1) Download data

2) Preprocess data (if necessary)

3) Define or load a configuration (config), which includes the following
   
   - task
   - model
   - optimizer
   - dataset
   - trainer

4) Train

5) Depending on the model/task there might be intermediate relaxation step

6) Predict

## Dataset

This examples uses the LMDB generated from the following [tutorial](http://laikapack.cheme.cmu.edu/notebook/open-catalyst-project/mshuaibi/notebooks/projects/ocp/docs/source/tutorials/lmdb_dataset_creation.ipynb). Please run that notebook before moving on. Alternatively, if you have other LMDBs available you may specify that instead.

In [2]:
# set the path to your local lmdb directory
train_src = "s2ef"

## Define config

For this example, we will explicitly define the config; however, a set of default config files exists in the config folder of this repository. Default config yaml files can easily be loaded with the `build_config` util (found in `ocp/ocpmodels/common/utils.py`). Loading a yaml config is preferrable when launching jobs from the command line. We have included our best models' config files [here](https://github.com/Open-Catalyst-Project/ocp/tree/master/configs/s2ef).

**Task** 

In [3]:
task = {
    'dataset': 'trajectory_lmdb', # dataset used for the S2EF task
    'description': 'Regressing to energies and forces for DFT trajectories from OCP',
    'type': 'regression',
    'metric': 'mae',
    'labels': ['potential energy'],
    'grad_input': 'atomic forces',
    'train_on_free_atoms': True,
    'eval_on_free_atoms': True
}

**Model** - SchNet for this example

In [4]:
model = {
    'name': 'schnet',
    'hidden_channels': 1024, # if training is too slow for example purposes reduce the number of hidden channels
    'num_filters': 256,
    'num_interactions': 3,
    'num_gaussians': 200,
    'cutoff': 6.0
}

**Optimizer**

In [5]:
optimizer = {
    'batch_size': 16, # if hitting GPU memory issues, lower this
    'eval_batch_size': 8,
    'num_workers': 8,
    'lr_initial': 0.0001,
    'lr_gamma': 0.1,
    'lr_milestones': [15, 20],
    'warmup_epochs': 10,
    'warmup_factor': 0.2,
    'max_epochs': 1, # used for demonstration purposes
    'force_coefficient': 100,
}

**Dataset**

For simplicity, `train_src` is used for all the train/val/test sets. Feel free to update with the actual S2EF val and test sets, but it does require additional downloads and preprocessing. If you desire to normalize your targets, `normalize_labels` must be set to `True` and corresponding `mean` and `stds` need to be specified. These values have been precomputed for you and can be found in any of the [`base.yml`](https://github.com/Open-Catalyst-Project/ocp/blob/master/configs/s2ef/20M/base.yml#L5-L9) config files.

In [6]:
dataset = [
{'src': train_src, 'normalize_labels': False}, # train set 
{'src': train_src}, # val set (optional)
{'src': train_src} # test set (optional - writes predictions to disk)
]

**Trainer**

Use the `ForcesTrainer` for the S2EF and IS2RS tasks, and the `EnergyTrainer` for the IS2RE task 

In [10]:
trainer = ForcesTrainer(
    task=task,
    model=model,
    dataset=dataset,
    optimizer=optimizer,
    identifier="SchNet-example",
    run_dir="./", # directory to save results if is_debug=False. Prediction files are saved here so be careful not to override!
    is_debug=False, # if True, do not save checkpoint, logs, or results
    is_vis=False,
    print_every=10,
    seed=0, # random seed to use
    logger="tensorboard", # logger of choice (tensorboard and wandb supported)
    local_rank=0,
    amp=False, # use PyTorch Automatic Mixed Precision (faster training and less memory usage)
)

amp: false
cmd:
  checkpoint_dir: ./checkpoints/2021-03-06-15-47-12-SchNet-example
  commit: 046e2fa
  identifier: SchNet-example
  logs_dir: ./logs/tensorboard/2021-03-06-15-47-12-SchNet-example
  print_every: 10
  results_dir: ./results/2021-03-06-15-47-12-SchNet-example
  seed: 0
  timestamp: 2021-03-06-15-47-12-SchNet-example
dataset:
  normalize_labels: false
  src: s2ef
gpus: 1
logger: tensorboard
logger_entity: null
logger_project: null
model: schnet
model_attributes:
  cutoff: 6.0
  hidden_channels: 1024
  num_filters: 256
  num_gaussians: 200
  num_interactions: 3
optim:
  batch_size: 16
  eval_batch_size: 8
  force_coefficient: 100
  lr_gamma: 0.1
  lr_initial: 0.0001
  lr_milestones:
  - 15
  - 20
  max_epochs: 1
  num_workers: 8
  warmup_epochs: 10
  warmup_factor: 0.2
task:
  dataset: trajectory_lmdb
  description: Regressing to energies and forces for DFT trajectories from OCP
  eval_on_free_atoms: true
  grad_input: atomic forces
  labels:
  - potential energy
  metric: 

## Check the model

In [11]:
print(trainer.model)

OCPDataParallel(
  (module): SchNet(hidden_channels=1024, num_filters=256, num_interactions=3, num_gaussians=200, cutoff=6.0)
)


## Train

In [12]:
trainer.train()

forcesx_mae: 0.0165, forcesy_mae: 0.0165, forcesz_mae: 1.7337, forces_mae: 0.5889, forces_cos: 0.0045, forces_magnitude: 1.7375, energy_mae: 10.1648, energy_force_within_threshold: 0.0000, loss: 69.0556, epoch: 0.0159
forcesx_mae: 0.0737, forcesy_mae: 0.0735, forcesz_mae: 0.6420, forces_mae: 0.2630, forces_cos: -0.0042, forces_magnitude: 0.7099, energy_mae: 3.0605, energy_force_within_threshold: 0.0000, loss: 29.3649, epoch: 0.1746
forcesx_mae: 0.0242, forcesy_mae: 0.0242, forcesz_mae: 0.3856, forces_mae: 0.1447, forces_cos: 0.0020, forces_magnitude: 0.4069, energy_mae: 0.9574, energy_force_within_threshold: 0.0000, loss: 15.4253, epoch: 0.3333
forcesx_mae: 0.0160, forcesy_mae: 0.0160, forcesz_mae: 0.1742, forces_mae: 0.0688, forces_cos: 0.0007, forces_magnitude: 0.1862, energy_mae: 0.3577, energy_force_within_threshold: 0.0000, loss: 7.2329, epoch: 0.4921
forcesx_mae: 0.0087, forcesy_mae: 0.0086, forcesz_mae: 0.1060, forces_mae: 0.0411, forces_cos: 0.0017, forces_magnitude: 0.1120, en

device 0: 100%|██████████| 126/126 [00:01<00:00, 86.02it/s]
/home/jovyan/miniconda3/envs/ocp-models/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


forcesx_mae: 0.0114, forcesy_mae: 0.0112, forcesz_mae: 0.1230, forces_mae: 0.0485, forces_cos: -0.0029, forces_magnitude: 0.1350, energy_mae: 0.3754, energy_force_within_threshold: 0.0000, loss: 5.2299, epoch: 1.0000
### Predicting on test.


device 0: 100%|██████████| 126/126 [00:01<00:00, 97.85it/s] 


Writing results to ./results/2021-03-06-15-47-12-SchNet-example/s2ef_predictions.npz


### Load Checkpoint
Once training has completed a `Trainer` class, by default, is loaded with the best checkpoint as determined by training or validation (if available) metrics. To load a `Trainer` class directly with a pretrained model, specify the `checkpoint_path` as defined by your previously trained model (`checkpoint_dir`):

In [19]:
checkpoint_path = os.path.join(trainer.config["cmd"]["checkpoint_dir"], "checkpoint.pt")
checkpoint_path

'./checkpoints/2021-03-06-15-47-12-SchNet-example/checkpoint.pt'

In [21]:
model = {
    'name': 'schnet',
    'hidden_channels': 1024, # if training is too slow for example purposes reduce the number of hidden channels
    'num_filters': 256,
    'num_interactions': 3,
    'num_gaussians': 200,
    'cutoff': 6.0
}

pretrained_trainer = ForcesTrainer(
    task=task,
    model=model,
    dataset=dataset,
    optimizer=optimizer,
    identifier="SchNet-example",
    run_dir="./", # directory to save results if is_debug=False. Prediction files are saved here so be careful not to override!
    is_debug=False, # if True, do not save checkpoint, logs, or results
    is_vis=False,
    print_every=10,
    seed=0, # random seed to use
    logger="tensorboard", # logger of choice (tensorboard and wandb supported)
    local_rank=0,
    amp=False, # use PyTorch Automatic Mixed Precision (faster training and less memory usage)
)

pretrained_trainer.load_pretrained(checkpoint_path=checkpoint_path)

amp: false
cmd:
  checkpoint_dir: ./checkpoints/2021-03-06-15-51-28-SchNet-example
  commit: 046e2fa
  identifier: SchNet-example
  logs_dir: ./logs/tensorboard/2021-03-06-15-51-28-SchNet-example
  print_every: 10
  results_dir: ./results/2021-03-06-15-51-28-SchNet-example
  seed: 0
  timestamp: 2021-03-06-15-51-28-SchNet-example
dataset:
  normalize_labels: false
  src: s2ef
gpus: 1
logger: tensorboard
logger_entity: null
logger_project: null
model: schnet
model_attributes:
  cutoff: 6.0
  hidden_channels: 1024
  num_filters: 256
  num_gaussians: 200
  num_interactions: 3
optim:
  batch_size: 16
  eval_batch_size: 8
  force_coefficient: 100
  lr_gamma: 0.1
  lr_initial: 0.0001
  lr_milestones:
  - 15
  - 20
  max_epochs: 1
  num_workers: 8
  warmup_epochs: 10
  warmup_factor: 0.2
task:
  dataset: trajectory_lmdb
  description: Regressing to energies and forces for DFT trajectories from OCP
  eval_on_free_atoms: true
  grad_input: atomic forces
  labels:
  - potential energy
  metric: 

/home/jovyan/miniconda3/envs/ocp-models/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:218: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


True

## Predict

If a test has been provided in your config, predictions are generated and written to disk automatically upon training completion. Otherwise, to make predictions on unseen data a `torch.utils.data` DataLoader object must be constructed. Here we reference our test set to make predictions on. Predictions are saved in `{results_file}.npz` in your `results_dir`.

In [22]:
# make predictions on the existing test_loader
predictions = pretrained_trainer.predict(pretrained_trainer.test_loader, results_file="s2ef_results", disable_tqdm=False)

### Predicting on test.


device 0: 100%|██████████| 126/126 [00:01<00:00, 93.22it/s] 

Writing results to ./results/2021-03-06-15-51-28-SchNet-example/s2ef_s2ef_results.npz


In [23]:
energies = predictions["energy"]
forces = predictions["forces"]